In [ ]:
!pip install streamlit pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

st.set_page_config(layout="wide")
st.title("🌤️ Weather Forecasting App")
st.markdown("Using OpenWeatherMap + Random Forest")

# --- Step 1: Get Data ---
API_KEY = "f2b6527500ebd13d7c2db06eee2c7654"
CITY = st.text_input("Enter City Name", "Mumbai")
URL = f"http://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={API_KEY}&units=metric"

@st.cache_data(show_spinner=False)
def fetch_weather():
    response = requests.get(URL)
    data = response.json()

    weather_data = []
    for item in data['list']:
        weather_data.append({
            'date_time': datetime.utcfromtimestamp(item['dt']),
            'temperature': item['main']['temp'],
            'humidity': item['main']['humidity'],
            'pressure': item['main']['pressure'],
            'wind_speed': item['wind']['speed'],
            'wind_direction': item['wind'].get('deg', 0),
            'cloud_coverage': item['clouds']['all'],
            'precipitation': item.get('rain', {}).get('3h', 0)
        })

    df = pd.DataFrame(weather_data)
    df = df.fillna(method='ffill')
    return df

weather_df = fetch_weather()

# --- Step 2: Model Training ---
X = weather_df.drop(['temperature', 'date_time'], axis=1)
y = weather_df['temperature']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestRegressor()
model.fit(X_train, y_train)

forecast_df = weather_df.copy()
forecast_df['predicted_temperature'] = model.predict(X)
mae = mean_absolute_error(y_test, model.predict(X_test))

# --- Step 3: Display Results ---
st.subheader("📊 Actual vs Predicted Temperature")
st.line_chart(forecast_df.set_index('date_time')[['temperature', 'predicted_temperature']])

st.markdown(f"### Mean Absolute Error: `{mae:.2f} °C`")

# --- Step 4: Explore Predictions ---
st.subheader("🔍 Explore Predictions")

row = st.slider("Select Time Index", 0, len(forecast_df) - 1, len(forecast_df) - 1)
data_point = forecast_df.iloc[row]

st.info(f"**Date/Time:** {data_point['date_time']}")
st.write(f"🌡️ **Actual Temp:** {data_point['temperature']} °C")
st.write(f"🔮 **Predicted Temp:** {data_point['predicted_temperature']:.2f} °C")

# --- Step 5: Feature Trend Visualization ---
st.subheader("📈 Feature Trend Visualization")
features = [col for col in weather_df.columns if col not in ['date_time', 'temperature']]
selected_features = st.multiselect("Select Features to Plot", features, default=[features[0]])

if selected_features:
    fig, ax = plt.subplots(figsize=(12, 6))
    for feature in selected_features:
        ax.plot(weather_df['date_time'], weather_df[feature], label=feature)
    ax.set_xlabel("Date Time")
    ax.set_ylabel("Value")
    ax.set_title("Weather Feature Trends")
    ax.legend()
    plt.xticks(rotation=45)
    st.pyplot(fig)


Writing app.py


In [ ]:
import os
from pyngrok import conf

# Replace YOUR_AUTHTOKEN with the token you copied
os.system("ngrok config add-authtoken 2vc4XCserTvHtl3itTxOd17cpf1_2aEgJGSifFavUoUUhQhKM")


0

In [ ]:
from pyngrok import ngrok
import time

# Kill previous tunnels
ngrok.kill()

# Start tunnel the new way
public_url = ngrok.connect("http://localhost:8501")
print(f"🌍 Your Streamlit app is live at: {public_url}")

# Run streamlit
!streamlit run app.py &>/content/logs.txt &
time.sleep(5)


🌍 Your Streamlit app is live at: NgrokTunnel: "https://ea1e-34-41-196-197.ngrok-free.app" -> "http://localhost:8501"
